In [12]:
from gensim import models
from pandas import DataFrame, concat
from os import path
from nltk.tokenize import word_tokenize

Загрузка данных и токенизация

In [9]:
with open(path.join('corpus', 'all_lem.txt')) as f:
    model_data = set(f.read().splitlines())

In [13]:
model_data = [word_tokenize(sentence) for sentence in model_data]

Обучение Word2Vec-модели

In [14]:
min_count = 5
size = 100
window = 10

model = models.Word2Vec(model_data, min_count=min_count, size=size, window=window,  workers=4)

Сериализация модели

In [15]:
from os import path, makedirs
from pickle import dump

subdir = 'models'
if not path.exists(subdir):
    makedirs(subdir)
file_path = path.join(subdir, '2ch_word2vec_all_lem')

model.save(file_path)

In [31]:
model.most_similar(morph.parse('черт')[0].normal_form)

[('характер', 0.8807381987571716),
 ('воображение', 0.8280839920043945),
 ('тон', 0.8147165775299072),
 ('взгляд', 0.8090890645980835),
 ('поведение', 0.8046504855155945),
 ('восприятие', 0.8008954524993896),
 ('проявление', 0.7976211309432983),
 ('склонный', 0.7954853773117065),
 ('эмоциональный', 0.7942137718200684),
 ('насилие', 0.7931546568870544)]